<a href="https://colab.research.google.com/github/akshatsinha0407/PythonColab/blob/main/Bidirectional_LSSTM_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fake NEws Classificer USsing Bidirectional LSTM

In [1]:
## Dataset: https://www.kaggle.com/competitions/fake-news/data?select=train.csv
import pandas as pd



In [4]:
df = pd.read_csv('train.csv', on_bad_lines='skip', engine='python')


In [5]:
df.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [6]:
df.isnull().sum()

,0
Unnamed: 0,0
title,126
text,29
label,20


In [7]:
df.shape

(19606, 4)

In [8]:
df=df.dropna()

In [9]:
df.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
5,5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1


In [10]:
## get the independent features
X=df.drop('label',axis=1)

In [46]:
X

,Unnamed: 0,title,text
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ..."
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will..."
5,5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...
...,...,...,...
19601,19581,"Upset at Trump, Mexico voices 'worry and irrit...",MEXICO CITY (Reuters) - Mexico on Thursday exp...
19602,19582,GOP Senator BURNS The Hell Out Of Trump With ...,GOP Senator Ben Sasse of Nebraska has been one...
19603,19583,FLYNN: Kathy Griffin Hopefully Decapitates Par...,Did you hear the one about comics? [Kathy Gr...
19604,19584,Trump Gives 6 Reasons To Vote For Him,"Thursday, 3 November 2016 'VOTE FOR ME, WHAT H..."


In [11]:
y=df['label']

In [47]:
y

,label
0,1
2,1
3,0
4,1
5,1
...,...
19601,0
19602,1
19603,0
19604,1


In [12]:
## Check whether dataset is balanced or not
y.value_counts()

,count
label,
1,10057
0,9397
is just unbelievably rich from North Sea oil.,1
за участие. Благодарю вас.,1


In [13]:
import tensorflow as tf

In [14]:
tf.__version__

'2.19.0'

In [16]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional


In [17]:
## vocabulary size
voc_size=5000

In [18]:
 messages=X.copy()

In [19]:
messages.head(10)

,Unnamed: 0,title,text
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ..."
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will..."
5,5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...
6,6,DR BEN CARSON TARGETED BY THE IRS: “I never ha...,DR. BEN CARSON TELLS THE STORY OF WHAT HAPPENE...
7,7,HOUSE INTEL CHAIR On Trump-Russia Fake Story: ...,
8,8,Sports Bar Owner Bans NFL Games…Will Show Only...,"The owner of the Ringling Bar, located south o..."
9,9,Latest Pipeline Leak Underscores Dangers Of Da...,"FILE – In this Sept. 15, 2005 file photo, the ..."
10,10,GOP Senator Just Smacked Down The Most Puncha...,The most punchable Alt-Right Nazi on the inter...


In [20]:
import nltk
import re
from nltk.corpus import stopwords

In [21]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [22]:
messages.reset_index(inplace=True)

In [26]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
14456
14457
14458
14459
14460
14461
14462
14463
14464
14465
14466
14467
14468
14469
14470
14471
14472
14473
14474
14475
14476
14477
14478
14479
14480
14481
14482
14483
14484
14485
14486
14487
14488
14489
14490
14491
14492
14493
14494
14495
14496
14497
14498
14499
14500
14501
14502
14503
14504
14505
14506
14507
14508
14509
14510
14511
14512
14513
14514
14515
14516
14517
14518
14519
14520
14521
14522
14523
14524
14525
14526
14527
14528
14529
14530
14531
14532
14533
14534
14535
14536
14537
14538
14539
14540
14541
14542
14543
14544
14545
14546
14547
14548
14549
14550
14551
14552
14553
14554
14555
14556
14557
14558
14559
14560
14561
14562
14563
14564
14565
14566
14567
14568
14569
14570
14571
14572
14573
14574
14575
14576
14577
14578
14579
14580
14581
14582
14583
14584
14585
14586
14587
14588
14589
14590
14591
14592
14593
14594
14595
14596
14597
14598
14599
14600
14601
14602
14603
14604
14605
14606
14607
14608
14609
14610
14611
14612
14613
1

In [27]:
corpus

['law enforc high alert follow threat cop white blacklivesmatt fyf terrorist video',
 'unbeliev obama attorney gener say charlott rioter peac protest home state north carolina video',
 'bobbi jindal rais hindu use stori christian convers woo evangel potenti bid',
 'satan russia unv imag terrifi new supernuk western world take notic',
 'time christian group sue amazon splc design hate group',
 'dr ben carson target ir never audit spoke nation prayer breakfast',
 'hous intel chair trump russia fake stori evid anyth video',
 'sport bar owner ban nfl game show true american sport like speak rural america video',
 'latest pipelin leak underscor danger dakota access pipelin',
 'gop senat smack punchabl alt right nazi internet',
 'may brexit offer would hurt cost eu citizen eu parliament',
 'schumer call trump appoint offici overse puerto rico relief',
 'watch hilari ad call question health age clinton crime famili boss',
 'chang expect espn polit agenda despit huge subscrib declin breitbart'

In [28]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[1598, 4089, 3296, 355, 2307, 2173, 4799, 2407, 4165, 1504, 3717, 690],
 [3108,
  2719,
  2216,
  2656,
  2590,
  660,
  2595,
  858,
  2269,
  4355,
  2115,
  4623,
  3549,
  690],
 [1888, 2121, 4794, 4146, 2190, 96, 227, 553, 2063, 517, 3348, 3335],
 [433, 2812, 1593, 4695, 2176, 3117, 4932, 2727, 4435, 1518, 2555],
 [4321, 227, 3986, 4655, 199, 4949, 3294, 4952, 3986],
 [357, 1309, 1668, 3509, 4873, 2166, 1901, 198, 2912, 4137, 2871],
 [1103, 3553, 977, 427, 2812, 4461, 96, 185, 2690, 690],
 [2907,
  1328,
  4497,
  433,
  1565,
  3728,
  3742,
  3651,
  3673,
  2907,
  311,
  671,
  794,
  4707,
  690],
 [3449, 1442, 155, 3391, 2261, 3753, 2650, 1442],
 [1901, 255, 4257, 1513, 3844, 447, 1729, 515],
 [2343, 1525, 804, 3955, 4626, 1461, 2764, 2375, 2764, 1022],
 [1489, 204, 427, 460, 1728, 2251, 3824, 3531, 2795],
 [4722, 625, 1398, 204, 1146, 230, 519, 4259, 888, 4567, 1125],
 [53, 333, 4083, 1647, 1583, 2931, 4569, 4734, 2910, 4182],
 [978, 1854, 4575, 1893, 352, 1103, 129],
 [36

## Embedding Representation

In [29]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

embedded_docs


array([[   0,    0,    0, ..., 1504, 3717,  690],
       [   0,    0,    0, ..., 4623, 3549,  690],
       [   0,    0,    0, ...,  517, 3348, 3335],
       ...,
       [   0,    0,    0, ..., 4939, 2257, 4182],
       [   0,    0,    0, ..., 2432, 1214, 4832],
       [   0,    0,    0, ..., 1913, 1369,  690]], dtype=int32)

In [30]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [31]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [32]:
## train test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [34]:
print(y_final[:10])
print(y_final.dtype)


['1' '1' '0' '1' '1' '1' '1' '1' '1' '1']
object


In [ ]:
for i, label in enumerate(y_final[:20]):
    print(i, repr(label))


In [56]:
import numpy as np

# Keep only rows where label is '0' or '1'
mask = np.isin(y_final, ['0','1'])
X_final = X_final[mask]
y_final = y_final[mask]

# Convert y_final to int
y_final = y_final.astype(int)



In [57]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_final, y_final, test_size=0.33, random_state=42
)


ValueError: With n_samples=0, test_size=0.33 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [55]:
model.fit(X_train, y_train,
          validation_data=(X_test, y_test),
          epochs=11,
          batch_size=32)


Epoch 1/11
408/408 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.9998 - loss: 0.0025 - val_accuracy: 0.8548 - val_loss: 1.1549
Epoch 2/11
408/408 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.9973 - loss: 0.0117 - val_accuracy: 0.8472 - val_loss: 1.0305
Epoch 3/11
408/408 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9985 - loss: 0.0068 - val_accuracy: 0.8481 - val_loss: 1.1573
Epoch 4/11
408/408 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9992 - loss: 0.0029 - val_accuracy: 0.8517 - val_loss: 1.2414
Epoch 5/11
408/408 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9997 - loss: 9.9079e-04 - val_accuracy: 0.8475 - val_loss: 1.2471
Epoch 6/11
408/408 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9996 - loss: 0.0015 - val_accuracy: 0.8514 - val_loss: 1.3659
Epoch 7/11
408/408 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9998 - loss: 9.3158e-04 - val_accuracy: 0.8500 - val_loss: 1.3676
Epoch 8/11
408/408 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9994 - loss: 0.0035 - va

In [58]:
y_pred=model.predict(X_test)

201/201 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [59]:
import numpy as np

y_pred=np.where(y_pred>=0.5,1,0)

In [60]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[2627,  448],
       [ 499, 2846]])

In [61]:
print(accuracy_score(y_test,y_pred))

0.8524922118380063
